In [ ]:
import pandas as pd
import os

def combine_ratings_to_single_parquet(input_paths, output_path):
    chunksize = 10**6
    all_data = []

    for input_path in input_paths:
        reader = pd.read_csv(
            input_path,
            header=None,
            names=["movie_id", "user_id", "date", "rating"],
            dtype={
                "movie_id": "int32",
                "user_id": "int32",
                "date": "string",
                "rating": "int8"
            },
            chunksize=chunksize
        )

        for chunk in reader:
            chunk["date"] = pd.to_datetime(chunk["date"], format="%Y-%m-%d")
            all_data.append(chunk)

    combined_df = pd.concat(all_data, ignore_index=True)

    combined_df.to_parquet(
        output_path,
        engine="pyarrow",
        compression="snappy",
        index=False
    )

input_paths = [f"data/rating_{n}.txt" for n in [1, 2, 3, 4]]
output_path = "prod_data/all_ratings.parquet"

combine_ratings_to_single_parquet(input_paths, output_path)
print("Exit 0")

-> not actually jupyter yet